<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argo.workflows.dsl import Workflow

from argo.workflows.dsl.tasks import *
from argo.workflows.dsl.templates import *

In [3]:
import yaml

from pprint import pprint

from argo.workflows.dsl._utils import sanitize_for_serialization

---

In [4]:
!sh -c '[ -f "artifacts.yaml" ] || curl -LO https://raw.githubusercontent.com/CermakM/argo-python-dsl/master/examples/artifacts.yaml'

In [5]:
from pathlib import Path

manifest = Path("./artifacts.yaml").read_text()
print(manifest)

# @file: artifacts.yaml
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  name: artifact-passing
  generateName: artifact-passing-
spec:
  entrypoint: main
  templates:
  - name: main
    dag:
      tasks:
      - name: generate-artifact
        template: whalesay
      - name: consume-artifact
        template: print-message
        arguments:
          artifacts:
          # bind message to the hello-art artifact
          # generated by the generate-artifact step
          - name: message
            from: "{{tasks.generate-artifact.outputs.artifacts.hello-art}}"

  - name: whalesay
    container:
      name: "whalesay"
      image: docker/whalesay:latest
      command: [sh, -c]
      args: ["cowsay hello world | tee /tmp/hello_world.txt"]
    outputs:
      artifacts:
      # generate hello-art artifact from /tmp/hello_world.txt
      # artifacts can be directories as well as files
      - name: hello-art
        path: /tmp/hello_world.txt

  - name: print-message
    inp

In [6]:
class ArtifactPassing(Workflow):
    
    @task
    def generate_artifact(self) -> V1alpha1Template:
        return self.whalesay()
    
    @task
    @artifact(
        name="message",
        _from="{{tasks.generate-artifact.outputs.artifacts.hello-art}}"
    )
    def consume_artifact(self, message: V1alpha1Artifact) -> V1alpha1Template:
        return self.print_message(message=message)
    
    @template
    @outputs.artifact(name="hello-art", path="/tmp/hello_world.txt")
    def whalesay(self) -> V1Container:
        container = V1Container(
            name="whalesay",
            image="docker/whalesay:latest",
            command=["sh", "-c"],
            args=["cowsay hello world | tee /tmp/hello_world.txt"]
        )
        
        return container
    
    @template
    @inputs.artifact(name="message", path="/tmp/message")
    def print_message(self, message: V1alpha1Artifact) -> V1Container:
        container = V1Container(
            name="print-message",
            image="alpine:latest",
            command=["sh", "-c"],
            args=["cat", "/tmp/message"],
        )
        
        return container

wf = ArtifactPassing()
wf

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generate_name': 'artifact-passing-', 'name': 'artifact-passing'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate-artifact',
                                            'template': 'whalesay'},
                                           {'arguments': {'artifacts': [{'_from': '{{tasks.generate-artifact.outputs.artifacts.hello-art}}',
                                                                         'name': 'message'}]},
                                            'name': 'consume-artifact',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'container': {'args': ['cowsay hello world | tee '
                                                '/tmp/hello_world.txt'],
                                       'command': ['sh', '-c'],
                                       'image

In [7]:
print(wf.to_yaml())

api_version: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generate_name: artifact-passing-
  name: artifact-passing
spec:
  entrypoint: main
  templates:
  - dag:
      tasks:
      - name: generate-artifact
        template: whalesay
      - arguments:
          artifacts:
          - _from: '{{tasks.generate-artifact.outputs.artifacts.hello-art}}'
            name: message
        name: consume-artifact
        template: print-message
    name: main
  - container:
      args:
      - cowsay hello world | tee /tmp/hello_world.txt
      command:
      - sh
      - -c
      image: docker/whalesay:latest
      name: whalesay
    name: whalesay
    outputs:
      artifacts:
      - name: hello-art
        path: /tmp/hello_world.txt
  - container:
      args:
      - cat
      - /tmp/message
      command:
      - sh
      - -c
      image: alpine:latest
      name: print-message
    inputs:
      artifacts:
      - name: message
        path: /tmp/message
    name: print-message
status

---

In [8]:
pprint(sanitize_for_serialization(wf))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'artifact-passing-', 'name': 'artifact-passing'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate-artifact',
                                            'template': 'whalesay'},
                                           {'arguments': {'artifacts': [{'from': '{{tasks.generate-artifact.outputs.artifacts.hello-art}}',
                                                                         'name': 'message'}]},
                                            'name': 'consume-artifact',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'container': {'args': ['cowsay hello world | tee '
                                                '/tmp/hello_world.txt'],
                                       'command': ['sh', '-c'],
                                       'image': 

In [9]:
pprint(yaml.safe_load(manifest))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'artifact-passing-', 'name': 'artifact-passing'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate-artifact',
                                            'template': 'whalesay'},
                                           {'arguments': {'artifacts': [{'from': '{{tasks.generate-artifact.outputs.artifacts.hello-art}}',
                                                                         'name': 'message'}]},
                                            'name': 'consume-artifact',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'container': {'args': ['cowsay hello world | tee '
                                                '/tmp/hello_world.txt'],
                                       'command': ['sh', '-c'],
                                       'image': 

In [10]:
from deepdiff import DeepDiff

diff = DeepDiff(sanitize_for_serialization(wf), yaml.safe_load(manifest))
diff

{}

In [11]:
assert not diff, "Manifests don't match."